In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. 选择并加载模型和 Tokenizer
model_name = r"C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 确保模型处于评估模式
model.eval()


c:\Users\k\.conda\envs\modelscope\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [2]:

# 2. 准备输入文本
prompt = "你是谁？"
input_ids = tokenizer.encode(prompt, return_tensors="pt") # "pt" 代表 PyTorch tensors

# 3. 执行单步推理（前向传播）
# 我们不需要计算梯度，所以使用 no_grad() 来节省计算资源并避免意外的训练。
with torch.no_grad():
    outputs = model(input_ids)
    # 'outputs' 包含多个部分，我们最关心的是 'logits'
    # logits 的形状通常是 [batch_size, sequence_length, vocab_size]
    # 我们想要的是 *下一个* Token 的预测，所以我们取序列中 *最后一个* Token 的 logits
    next_token_logits = outputs.logits[:, -1, :] # 形状变为 [batch_size, vocab_size]


In [4]:

# 4. 获取候选 Token 及其分数
# 我们可以获取 Logit 值最高的 K 个 Token

k = 10 # 我们想看 Top 10 的候选者

# (a) 获取 Top-K Logits
top_k_logits, top_k_indices = torch.topk(next_token_logits, k)
top_k_logits, top_k_indices

(tensor([[14.7066, 14.6095, 14.1463, 14.1403, 13.7114, 13.2062, 13.0800, 12.9611,
          12.7823, 12.7142]]),
 tensor([[ 35946,  56568, 104198, 105043,  97639,   9909,    220,  49434,  18493,
          104786]]))

In [6]:

# (b) (可选) 将 Logits 转换为概率
# 使用 Softmax 函数将 Logits 转换为概率分布
probabilities = torch.nn.functional.softmax(next_token_logits, dim=-1)
probabilities

tensor([[3.9805e-05, 8.3852e-04, 2.6764e-05,  ..., 8.1092e-10, 8.1154e-10,
         8.1090e-10]])

In [8]:
top_k_probs, top_k_indices_probs = torch.topk(probabilities, k) # 这里的 indices 应该和上面一样
top_k_probs, top_k_indices_probs

(tensor([[0.1209, 0.1097, 0.0690, 0.0686, 0.0447, 0.0270, 0.0238, 0.0211, 0.0176,
          0.0165]]),
 tensor([[ 35946,  56568, 104198, 105043,  97639,   9909,    220,  49434,  18493,
          104786]]))

In [9]:

# 5. 打印结果
# 将 Token 索引解码回文本
top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices[0]) # [0] 是因为 batch_size=1
top_k_tokens

['æĪĳ',
 'ä½ł',
 'æĪĳæĺ¯',
 'ä½łæĺ¯',
 'æĪĳä»¬',
 'ï¼Ī',
 'Ġ',
 'ĠæĪ',
 'åľ¨',
 'æĪĳåľ¨']

In [10]:
top_k_tokens_probs = tokenizer.convert_ids_to_tokens(top_k_indices_probs[0])
top_k_tokens_probs

['æĪĳ',
 'ä½ł',
 'æĪĳæĺ¯',
 'ä½łæĺ¯',
 'æĪĳä»¬',
 'ï¼Ī',
 'Ġ',
 'ĠæĪ',
 'åľ¨',
 'æĪĳåľ¨']

In [16]:

print(f"输入文本: '{prompt}'")
print("-" * 30)
print(f"模型预测的 Top-{k} 个下一个 Token (基于 Logits):")

for token, logit in zip(top_k_tokens, top_k_logits[0]):
    # 使用 .item() 从 Tensor 中提取 Python 数字
    print(f"  - Token: '{token.replace('Ġ', ' ')}', Logit: {logit.item():.4f}") # 'Ġ' 通常代表空格


输入文本: '你是谁？'
------------------------------
模型预测的 Top-10 个下一个 Token (基于 Logits):
  - Token: 'æĪĳ', Logit: 14.7066
  - Token: 'ä½ł', Logit: 14.6095
  - Token: 'æĪĳæĺ¯', Logit: 14.1463
  - Token: 'ä½łæĺ¯', Logit: 14.1403
  - Token: 'æĪĳä»¬', Logit: 13.7114
  - Token: 'ï¼Ī', Logit: 13.2062
  - Token: ' ', Logit: 13.0800
  - Token: ' æĪ', Logit: 12.9611
  - Token: 'åľ¨', Logit: 12.7823
  - Token: 'æĪĳåľ¨', Logit: 12.7142


In [14]:
print("-" * 30)
print(f"模型预测的 Top-{k} 个下一个 Token (基于概率):")

for token, prob in zip(top_k_tokens_probs, top_k_probs[0]):
    # 解码单个token为可读文本
    decoded_token = tokenizer.decode([tokenizer.convert_tokens_to_ids(token)])
    print(f"  - Token: '{token}' -> 解码: '{decoded_token}', 概率: {prob.item():.4%}")


------------------------------
模型预测的 Top-10 个下一个 Token (基于概率):
  - Token: 'æĪĳ' -> 解码: '我', 概率: 12.0899%
  - Token: 'ä½ł' -> 解码: '你', 概率: 10.9720%
  - Token: 'æĪĳæĺ¯' -> 解码: '我是', 概率: 6.9037%
  - Token: 'ä½łæĺ¯' -> 解码: '你是', 概率: 6.8625%
  - Token: 'æĪĳä»¬' -> 解码: '我们', 概率: 4.4693%
  - Token: 'ï¼Ī' -> 解码: '（', 概率: 2.6967%
  - Token: 'Ġ' -> 解码: ' ', 概率: 2.3769%
  - Token: 'ĠæĪ' -> 解码: ' �', 概率: 2.1105%
  - Token: 'åľ¨' -> 解码: '在', 概率: 1.7649%
  - Token: 'æĪĳåľ¨' -> 解码: '我在', 概率: 1.6487%


In [13]:

# 6. (可选) 演示如何选择一个 Token 并继续生成
chosen_token_id = top_k_indices[0][0].unsqueeze(0).unsqueeze(0) # 选择 Logit 最高的那个
new_input_ids = torch.cat([input_ids, chosen_token_id], dim=-1)
new_prompt = tokenizer.decode(new_input_ids[0])

print("-" * 30)
print(f"如果选择 Logit 最高的 Token，下一个输入将是: '{new_prompt}'")

------------------------------
如果选择 Logit 最高的 Token，下一个输入将是: '你是谁？我'
